In [1]:
import numpy as np
import pandas as pd

In [2]:
def url(latitude, longitude, angle):
    '''
    Form the full URL to access data via the PVGIS API.

    Parameters:
        latitude: {float}
            Latitude of the location in decimal degrees.
        longitude: {float}
            Longitude of the location in decimal degrees.
        angle: {float}
            tilt angle for irradiation.
    Returns:
        {string}
    '''

    base_url = 'https://re.jrc.ec.europa.eu/api/v5_2/MRcalc' # The base url to access data via the PVGIS API

    start_year = 2005 # First year for the desire outcome
    end_year = 2020 # Final year for the desire outcome

    parameters = {'lat': latitude,
                  'lon': longitude,
                  'raddatabase': 'PVGIS-ERA5', # Name of the radiation database to be used according to the chosen location
                  'startyear': start_year,
                  'endyear': end_year,
                  'outputformat': 'csv', # Type of output: comma separated values (csv)
                  'angle': angle,
                  'selectrad': 1, # Irradiation on plane of selected inclination
                  'avtemp': 1, # Monthly average values of daily (24h) temperature
                  'd2g': 1, # Ratio of diffuse to global radiation (horizontal plane)
                  'mr_dni': 1, # Direct normal irradiation
                  'horirrad': 1, # Horizontal plane irradiation
                 }

    query_string = '&'.join([f'{key}={parameters[key]}' for key in parameters]) # Forming the query string

    full_url = base_url + '?' + query_string #The full url to access data via the PVGIS API

    return full_url

In [3]:
# Loading Peruvian city data
cities = pd.read_csv('peruvian_cities.csv', encoding = "ISO-8859-1", index_col= 'city_index')
cities

,city,department,latitude,longitude
city_index,,,,
1,Tumbes,Tumbes,-3.570833,-80.459722
2,Piura,Piura,-5.200833,-80.625278
3,Chiclayo,Lambayeque,-6.762961,-79.836614
4,Trujillo,La Libertad,-8.111944,-79.028889
5,Huaraz,Áncash,-9.533333,-77.533333
6,Lima,Lima,-12.060000,-77.037500
7,Ica,Ica,-14.066667,-75.733333
8,Arequipa,Arequipa,-16.398764,-71.536883
9,Moquegua,Moquegua,-17.200000,-70.933333


In [4]:
# Tilt angles of solar panels
angles = range(0, 91)

In [5]:
# Getting solar data for Peruvian cities at different tilt angles
all_data = []

for index in cities.index:
  lat = cities.loc[index, 'latitude']
  lon = cities.loc[index, 'longitude']

  for angle in angles:
    solar_data = pd.read_csv(url(lat, lon, angle), header = 3, skipfooter = 7, sep = '\t\t', engine = 'python') # Gets the solar data
    solar_data['angle'] = angle
    solar_data['city_index'] = index
    all_data.append(solar_data)

all_solar_data = pd.concat(all_data, axis = 0, ignore_index = True)
all_solar_data.to_csv('solar_data.csv', index = False)